In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from datetime import  timedelta

In [2]:
products = pd.read_csv('C:/Users/sepid/Desktop/H & m/articles.csv')

In [66]:
products['index_group_name'].unique()

array(['Ladieswear', 'Baby/Children', 'Menswear', 'Sport', 'Divided'],
      dtype=object)

# unknown entries in germent group name column

In [4]:
products['garment_group_name'].value_counts()

Jersey Fancy                     21445
Accessories                      11519
Jersey Basic                      8126
Knitwear                          7490
Under-, Nightwear                 7441
Trousers                          6727
Blouses                           5838
Shoes                             5145
Dresses Ladies                    4874
Outdoor                           4501
Unknown                           3815
Trousers Denim                    3100
Swimwear                          2822
Socks and Tights                  2295
Shirts                            2116
Woven/Jersey/Knitted mix Baby     1965
Shorts                            1559
Dresses/Skirts girls              1541
Skirts                            1254
Special Offers                    1061
Dressed                            908
Name: garment_group_name, dtype: int64

In [5]:
products[products['garment_group_name']=='Unknown']['product_group_name'].value_counts()

Garment Upper body       1934
Garment Lower body       1041
Garment Full body         543
Accessories               176
Underwear                  63
Shoes                      29
Furniture                  13
Garment and Shoe care       9
Stationery                  5
Unknown                     2
Name: product_group_name, dtype: int64

for very small amount, it can be found out what unknown is by checking product group name. for example unknown where it has sweamwear product name then it is sweamwear in germant also because we only have on sweamwear type. it goes same for Socks & Tights also

In [3]:
products.loc[(products['garment_group_name'] == 'Unknown') & (products['product_group_name'] == 'Swimwear'), 'garment_group_name'] = 'Swimwear'
products.loc[(products['garment_group_name'] == 'Unknown') & (products['product_group_name'] == 'Socks & Tights'), 'garment_group_name'] = 'Socks and Tights'

this section is the same for weighted or non weighted probability. In this part every article has given a quotient score.

In [3]:
df_org  = pd.read_csv('C:/Users/sepid/Desktop/H & m/transactions_train.csv',
                        usecols= ['t_dat', 'customer_id', 'article_id'], 
                        dtype={'article_id': 'int32', 't_dat': 'string', 'customer_id': 'string'})
df_org['t_dat'] = pd.to_datetime(df_org['t_dat'])
df_org1 = df_org[df_org['t_dat']>='2019-01-01'] 

In [4]:
df = df_org1.copy()
df = df.merge(products[['article_id', 'index_group_name']], on='article_id', how='inner')

df['customer_id'] = df['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

# Find the maximum purchase date in the dataset which is 2020-09-22 and it is Tuesday
last_ts = df['t_dat'].max()

# Calculate weekly counts of purchases starting from every Tuesday to the next Tuesday
# Create a temporary DataFrame to store the date and its day of the week
tmp = df[['t_dat']].copy()
tmp['dow'] = tmp['t_dat'].dt.dayofweek

# Calculate the last Tuesday's date for each purchase date
# for example if it is Thursday it will return 2 days before and if it is Monday it will return date of tomorrow
tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')

# calculating the next Tuesday 
tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

# Add the 'ldbw' column to df
df['ldbw'] = tmp['ldbw'].values

# Count the number of purchases for each product and week (grouping products with lbdw)
weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id', 'index_group_name']).count().reset_index()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})

# Merge the weekly sales counts back into the main DataFrame
df = df.merge(weekly_sales, on=['ldbw', 'article_id', 'index_group_name'], how = 'left')

# Reset and set the index for 'article_id' for later use
weekly_sales = weekly_sales.reset_index().set_index('article_id')

# Count the amount of last week's purchases for each product 
df = df.merge(
    weekly_sales.loc[weekly_sales['ldbw']==last_ts, ['count']],
    on='article_id', suffixes=("", "_targ"))

# Fill missing values with 0 for products that had no purchases in the last week
df['count_targ'].fillna(0, inplace=True)
del weekly_sales

# Calculate the quotient for each product, which is the last week's purchases divided by each week's purchases
df['quotient'] = df['count_targ'] / df['count']

# then all quotients are summed for each article individually
target_sales = df.drop('customer_id', axis=1).groupby(['article_id', 'index_group_name'])['quotient'].sum()
# this target sales is each product score and based on that the top 12 can be recommended to the customers with no purchase history
general_pred = target_sales.nlargest(12).index.tolist()
general_pred = ['0' + str(article_id) for article_id in general_pred]
general_pred_str =  ' '.join(general_pred)

In [10]:
target_sales

article_id  garment_group_name
108775044   Jersey Basic           315.0
111565001   Socks and Tights      3016.0
111586001   Socks and Tights      4851.0
111593001   Socks and Tights      4059.0
111609001   Socks and Tights      1260.0
                                   ...  
952267001   Socks and Tights        40.0
952938001   Jersey Fancy            14.0
953450001   Socks and Tights        15.0
953763001   Jersey Fancy            36.0
956217002   Jersey Fancy            16.0
Name: quotient, Length: 17986, dtype: float64

# using non weighted popularity (only quotient)

In [5]:
#  every index_group_name is being ranked based on its quotient score
target_sales = pd.DataFrame(target_sales).reset_index()
target_sales['rank'] = target_sales.groupby("index_group_name")['quotient'].rank("dense", ascending=False)

In [13]:
target_sales[target_sales['index_group_name']=='Ladieswear'].sort_values(by=['index_group_name'],ascending=True)

,article_id,index_group_name,quotient,rank
0,108775044,Ladieswear,273.0,770.0
12476,871957004,Ladieswear,117.0,891.0
12477,871974001,Ladieswear,52.0,950.0
12480,871992001,Ladieswear,20.0,982.0
12481,871992002,Ladieswear,180.0,840.0
...,...,...,...,...
6762,802023001,Ladieswear,600.0,565.0
6763,802023002,Ladieswear,936.0,425.0
6764,802023003,Ladieswear,88.0,914.0
6766,802023005,Ladieswear,720.0,511.0


In [6]:
#  for every group rank only 12 highest ones are chosen
target_sales = target_sales[target_sales['rank'] <= 12]

so for every group we have 12 most popular articles

In [7]:
target_sales[target_sales['index_group_name']=='Ladieswear']

,article_id,index_group_name,quotient,rank
29,158340001,Ladieswear,21930.0,5.0
271,372860001,Ladieswear,21476.0,6.0
272,372860002,Ladieswear,21240.0,7.0
1065,562245046,Ladieswear,18473.0,9.0
1269,573085028,Ladieswear,33670.0,2.0
1361,579541001,Ladieswear,22113.0,4.0
1647,610776002,Ladieswear,17836.0,10.0
2374,673677002,Ladieswear,24512.0,3.0
2467,678942001,Ladieswear,17100.0,12.0
4496,751471001,Ladieswear,34716.0,1.0


In [11]:
# calculating for every customer probability of buying any specific germant(index) group
rr = (df.groupby(['customer_id', 'index_group_name'])['article_id'].agg(['count']) / df.groupby(['customer_id'])['article_id'].agg(['count']) ).reset_index()
rr = rr.merge(target_sales, on='index_group_name')
# count here is probability of that customer purchasing that index group article

,customer_id,index_group_name,count
0,-9223352921020755230,Divided,0.428571
1,-9223352921020755230,Ladieswear,0.571429
2,-9223343869995384291,Divided,0.067797
3,-9223343869995384291,Ladieswear,0.728814
4,-9223343869995384291,Menswear,0.118644
...,...,...,...
1982389,9223301031927667525,Divided,0.500000
1982390,9223301031927667525,Ladieswear,0.500000
1982391,9223333063893176977,Divided,1.000000
1982392,9223357421094039679,Ladieswear,1.000000


In [15]:
rr[rr['customer_id']==-9223343869995384291]

,customer_id,index_group_name,count
2,-9223343869995384291,Divided,0.067797
3,-9223343869995384291,Ladieswear,0.728814
4,-9223343869995384291,Menswear,0.118644
5,-9223343869995384291,Sport,0.084746


In [29]:
# multiplying the probability with the article score to get new ranking
rr['new_rank'] = rr['rank'] * (1 - rr['count'])

In [ ]:
rr['new_rank1'] = rr.groupby("customer_id")['new_rank'].rank('first', ascending=True, )

In [16]:
rr = rr.loc[rr['new_rank1'] <= 12]
rr

,customer_id,index_group_name,count,article_id,quotient,rank,new_rank,new_rank1
0,-9223352921020755230,Divided,0.428571,448509014,44100.0,1.0,0.571429,2.0
3,-9223352921020755230,Divided,0.428571,706016001,29939.0,2.0,1.142857,4.0
4,-9223352921020755230,Divided,0.428571,706016002,21931.0,5.0,2.857143,11.0
5,-9223352921020755230,Divided,0.428571,706016003,25844.0,4.0,2.285714,9.0
8,-9223352921020755230,Divided,0.428571,715624001,26712.0,3.0,1.714286,6.0
...,...,...,...,...,...,...,...,...
23788723,9221454157560484085,Baby/Children,1.000000,594541019,1260.0,1.0,0.000000,8.0
23788724,9221454157560484085,Baby/Children,1.000000,659808003,492.0,8.0,0.000000,9.0
23788725,9221454157560484085,Baby/Children,1.000000,703279001,520.0,6.0,0.000000,10.0
23788726,9221454157560484085,Baby/Children,1.000000,731520001,476.0,9.0,0.000000,11.0


In [ ]:
purchase_df = rr.sort_values(['customer_id', 'new_rank1'], ascending = False).reset_index(drop = True)
purchase_df['prediction'] = '0' + purchase_df['article_id'].astype(str) + ' '
purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
purchase_df['prediction'] = purchase_df['prediction'].str.strip()
purchase_df = pd.DataFrame(purchase_df)

# Read the submission file
sub  = pd.read_csv('C:/Users/sepid/Desktop/H & m/sample_submission.csv',
                    usecols= ['customer_id'], 
                    dtype={'customer_id': 'string'})
sub['customer_id2'] = sub['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
sub = sub.merge(purchase_df, left_on = 'customer_id2', right_on = 'customer_id', how = 'left',
           suffixes = ('', '_ignored'))

# Customers with no purchase history will be recommended with the general prediction (top 12 quotients)
sub['prediction'] = sub['prediction'].fillna(general_pred_str)

# For customers with existing purchase history, if their number of predictions is less than 12,
# the rest will be filled with general predictions
sub['prediction'] = sub['prediction'] + ' ' +  general_pred_str
sub['prediction'] = sub['prediction'].str.strip()
sub['prediction'] = sub['prediction'].str[:131]
sub = sub[['customer_id', 'prediction']]
sub.to_csv('C:/Users/sepid/Desktop/H & m/submission_.csv',index=False)
# this has public score (for index group) of 0.00381 and private score of 0.0037

# index grouping with weighted popularity

in this part I calculated weighted weekly popularity of each index and also probability of each customer buying each index

In [18]:
a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
tmp = df.copy()

# Calculate the days difference between the last date and the purchase date
tmp['x'] = ((last_ts - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)

# If it's the last date of purchase, substitute 0 for 'x' with 1
tmp['dummy_1'] = 1 
tmp['x'] = tmp[["x", "dummy_1"]].max(axis=1)

# Calculate a weight ('y') based on the time difference ('x')
tmp['y'] = a / np.sqrt((tmp['x']+1)) + b * np.exp(-c*(tmp['x']+1)) - d

# Substitute negative weights with 0
tmp['dummy_0'] = 0 
tmp['y'] = tmp[["y", "dummy_0"]].max(axis=1)

# Multiply weights by quotients and creating new score "value"
tmp['value'] = tmp['quotient'] * tmp['y']

# values are agreggated for every article
tmp = tmp.groupby(['article_id','index_group_name']).agg({'value': 'sum'})
tmp = tmp.reset_index()

# Rank the values and select the top 12 recommendations for each index_group_name
tmp = tmp.loc[tmp['value'] > 0]
tmp['rank'] = tmp.groupby("index_group_name")['value'].rank("dense", ascending=False)
tmp = tmp[tmp['rank'] <= 12]

In [22]:
tmp[tmp['index_group_name']=='Ladieswear']

,article_id,index_group_name,value,rank
1269,573085028,Ladieswear,6.786626e+07,7.0
1273,573085042,Ladieswear,6.314768e+07,9.0
2374,673677002,Ladieswear,6.049670e+07,11.0
4496,751471001,Ladieswear,9.214537e+07,2.0
5030,762846027,Ladieswear,5.767777e+07,12.0
10156,850917001,Ladieswear,7.133185e+07,4.0
16773,915529003,Ladieswear,6.532041e+07,8.0
16774,915529005,Ladieswear,6.126290e+07,10.0
16961,918522001,Ladieswear,6.960707e+07,6.0
17287,923758001,Ladieswear,7.032722e+07,5.0


In [24]:
# in this part again probability of purchase of each index group is calculated
prob = (df.groupby(['customer_id', 'index_group_name'])['article_id'].agg(['count']) / df.groupby(['customer_id'])['article_id'].agg(['count']) ).reset_index()
prob = prob.merge(tmp, on='index_group_name')

# count here is probability of that customer purchasing that index group article
prob['new_rank'] = prob['rank'] * (1 - prob['count'])

# new_rank will be ranked as new_rank1
prob['new_rank1'] = prob.groupby("customer_id")['new_rank'].rank('first', ascending=True, )

# only 12 highest are chosen
prob = prob.loc[prob['new_rank1'] <= 12]

In [25]:
prob[prob['customer_id']== -9223352921020755230]

,customer_id,index_group_name,count,article_id,value,rank,new_rank,new_rank1
0,-9223352921020755230,Divided,0.428571,448509014,8.829856e+07,1.0,0.571429,2.0
1,-9223352921020755230,Divided,0.428571,706016001,5.693169e+07,3.0,1.714286,6.0
4,-9223352921020755230,Divided,0.428571,714790020,8.237505e+07,2.0,1.142857,4.0
5,-9223352921020755230,Divided,0.428571,715624001,5.473250e+07,4.0,2.285714,9.0
6,-9223352921020755230,Divided,0.428571,788575002,5.431679e+07,5.0,2.857143,11.0
7369188,-9223352921020755230,Ladieswear,0.571429,573085028,6.786626e+07,7.0,3.000000,12.0
7369191,-9223352921020755230,Ladieswear,0.571429,751471001,9.214537e+07,2.0,0.857143,3.0
7369193,-9223352921020755230,Ladieswear,0.571429,850917001,7.133185e+07,4.0,1.714286,7.0
7369196,-9223352921020755230,Ladieswear,0.571429,918522001,6.960707e+07,6.0,2.571429,10.0
7369197,-9223352921020755230,Ladieswear,0.571429,923758001,7.032722e+07,5.0,2.142857,8.0


In [26]:
purchase_df = prob.sort_values(['customer_id', 'new_rank1'], ascending = False).reset_index(drop = True)
purchase_df['prediction'] = '0' + purchase_df['article_id'].astype(str) + ' '
purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
purchase_df['prediction'] = purchase_df['prediction'].str.strip()
purchase_df = pd.DataFrame(purchase_df)

# Read the submission file
sub  = pd.read_csv('C:/Users/sepid/Desktop/H & m/sample_submission.csv',
                    usecols= ['customer_id'], 
                    dtype={'customer_id': 'string'})
sub['customer_id2'] = sub['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
sub = sub.merge(purchase_df, left_on = 'customer_id2', right_on = 'customer_id', how = 'left',
           suffixes = ('', '_ignored'))

# Customers with no purchase history will be recommended with the general prediction (top 12 quotients)
sub['prediction'] = sub['prediction'].fillna(general_pred_str)

# For customers with existing purchase history, if their number of predictions is less than 12,
# the rest will be filled with general predictions
sub['prediction'] = sub['prediction'] + ' ' +  general_pred_str
sub['prediction'] = sub['prediction'].str.strip()
sub['prediction'] = sub['prediction'].str[:131]
sub = sub[['customer_id', 'prediction']]
sub.to_csv('C:/Users/sepid/Desktop/H & m/submission_.csv',index=False)
# this has public score of 0.00574 and private score of 0.00577

# product index combined with age group popularity

In this section, article group popularity will be calculated based on each age group. then as previous works, probability of purchasing each product group will be calculated for customers then these probabilities going to be multiplied by pruduct group popularities and create new ranks.

doing all steps done before for customer transformation and age group creation

In [17]:
customers  = pd.read_csv('C:/Users/sepid/Desktop/H & m/customers.csv')
customers = customers.drop(['postal_code'], axis=1)
customers['FN'] = customers['FN'].fillna(0)
customers['Active'] = customers['Active'].fillna(0)
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace({np.nan :0,'NONE':0, 'None':0, 'Monthly':1, 'Regularly':2})
customers['club_member_status'] = customers['club_member_status'].replace({np.nan :0, 'PRE-CREATE':1, 'ACTIVE':2, 'LEFT CLUB':3})
customers['group'] = customers['age'].apply(lambda x : 1 if x<30 else(2 if 30 <= x < 50 else 3))
customers = customers.drop(['FN','Active','club_member_status','fashion_news_frequency','age'], axis=1)

In [86]:
customers

,customer_id,group
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,1
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,1
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,3
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,3
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,1
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,1
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,1
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1


In [20]:
listUniBins = customers['group'].unique()

In [18]:
# creating product group probability for each customer
m = df_org1.merge(products[['index_group_name','article_id']], on='article_id', how='inner')
m['customer_id'] = m['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
index_rank = (m.groupby(['customer_id', 'index_group_name'])['article_id'].agg(['count']) / m.groupby(['customer_id'])['article_id'].agg(['count'])).reset_index()
index_rank = index_rank.rename(columns={'count': 'probability'})

In [33]:
index_rank

,customer_id,garment_group_name,probability
0,-9223352921020755230,Accessories,0.100000
1,-9223352921020755230,Blouses,0.200000
2,-9223352921020755230,Knitwear,0.200000
3,-9223352921020755230,Outdoor,0.100000
4,-9223352921020755230,Trousers,0.400000
...,...,...,...
6974249,9223357421094039679,Blouses,0.192308
6974250,9223357421094039679,Jersey Basic,0.769231
6974251,9223357421094039679,Jersey Fancy,0.038462
6974252,9223370729206611574,Jersey Fancy,0.500000


In [34]:
index_rank[index_rank['customer_id'] == 9223333063893176977]

,customer_id,garment_group_name,probability
6974246,9223333063893176977,Blouses,0.333333
6974247,9223333063893176977,Socks and Tights,0.333333
6974248,9223333063893176977,Trousers Denim,0.333333


for every age group I created a 40 article popularity as the baseline

In [21]:
a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
gen_1=[]
gen_2=[]
gen_3=[]
for uniBin in listUniBins: 
    
    df = df_org1.copy()
    dfCustomersTemp = customers[customers['group'] == uniBin]
    dfCustomersTemp = dfCustomersTemp.drop(['group'], axis=1)
    
    #
    df = df.merge(dfCustomersTemp[['customer_id']], on='customer_id', how='inner')
    print(f'The shape of scope transaction for {uniBin} is {df.shape}. \n')

    # transformation part
    df['customer_id'] = df['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
    # here we get the maximum date of purchase we have which is 2020-09-22 and it is Tuesday
    last_ts = df['t_dat'].max()
    # as the last date is Tuesday so weekly count of purchases are going to be counted from every Tuesday to the next Tuesday
    # for doing so first we nee dto have all dates and their day of week
    tmp = df[['t_dat']].copy()
    tmp['dow'] = tmp['t_dat'].dt.dayofweek
    #in this part every t-dat is taken and find the Tuesday related to the week of that t-dat and saves it in lbdw
    # for example if date is Thursday it will return date of 2 days ago which is Tuesday and if it is Monday it will return date of tomorrow
    tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
    #next step is calculating next Tuesday after the one in lbdw
    tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')
    df['ldbw'] = tmp['ldbw'].values
    #so basically here we count purchase of articles for each week (grouping products with lbdw)
    weekly_sales = df.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
    weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
    df = df.merge(weekly_sales, on=['ldbw', 'article_id'], how = 'left')
    weekly_sales = weekly_sales.reset_index().set_index('article_id')
    #count_targ is amount of last week purchase of each product    
    df = df.merge(
        weekly_sales.loc[weekly_sales['ldbw']==last_ts, ['count']],
        on='article_id', suffixes=("", "_targ"))
    # in case of no last week purchase of that product it will be filled with 0
    df['count_targ'].fillna(0, inplace=True)
    del weekly_sales
    # based on last week purchase a quotient is calculated which is divisison of amouunt of last week purchase over each week
    df['quotient'] = df['count_targ'] / df['count']
    # then all quotients are summed for each article individually
    # thisn qoutient can be used as a baseline also
    target_sales = df.drop('customer_id', axis=1).groupby('article_id', as_index=False)['quotient'].sum()
    target_sales['rank'] = target_sales['quotient'].rank(ascending=False, method='dense')
    target_sales = target_sales[target_sales['rank'] <= 12]
    
    general_pred = target_sales['article_id'].tolist()
    general_pred = ['0' + str(article_id) for article_id in general_pred]
    if uniBin==1:
        gen_1 =  ' '.join(general_pred)
    elif uniBin==2:
        gen_2 =  ' '.join(general_pred)
    else:
        gen_3 =  ' '.join(general_pred)
#     target_sales.to_csv(f'C:/Users/sepid/Desktop/H & m/general/' + str(uniBin) + '.csv',index=False)
    del target_sales
    
    tmp = df.copy()
    # here days difference between last date and purchase date of each transiction is calculated
    tmp['x'] = ((last_ts - tmp['t_dat']) / np.timedelta64(1, 'D')).astype(int)
    # if it it the last date purchase then we substitude 0 x with 1
    tmp['dummy_1'] = 1 
    tmp['x'] = tmp[["x", "dummy_1"]].max(axis=1)
    #by using this function, transaction dates closer to last week are given more importance
    tmp['y'] = a / np.sqrt((tmp['x']+1)) + b * np.exp(-c*(tmp['x']+1)) - d
    # substitution of negative weights by 0
    tmp['dummy_0'] = 0 
    tmp['y'] = tmp[["y", "dummy_0"]].max(axis=1)
    # wight are multiplied by qoutients and then aggregated for each customer and article
    tmp['value'] = tmp['quotient'] * tmp['y'] 
    tmp = tmp.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
    tmp = tmp.reset_index()
    # we rank the value and then 12 highest ranks for each customer
    tmp = tmp.loc[tmp['value'] > 0]
    tmp['rank'] = tmp.groupby("customer_id")["value"].rank("first", ascending=False)
    tmp = tmp.loc[tmp['rank'] <= 40]
    
    
    tmp.to_csv(f'C:/Users/sepid/Desktop/H & m/pred/' + str(uniBin) + '.csv',index=False)



The shape of scope transaction for 2 is (9618849, 3). 

The shape of scope transaction for 1 is (12025918, 3). 

The shape of scope transaction for 3 is (5732295, 3). 



In [24]:
gen_3

'0448509014 0562245046 0573085028 0579541001 0673677002 0678942001 0706016001 0714790020 0751471001 0762846006 0768912001 0783346001'

In [22]:
file_paths = ['C:/Users/sepid/Desktop/H & m/pred/1.csv',
              'C:/Users/sepid/Desktop/H & m/pred/2.csv',
              'C:/Users/sepid/Desktop/H & m/pred/3.csv']
dfs = []
header = pd.read_csv(file_paths[0], nrows=0).columns.tolist()


for file_path in file_paths:
    df = pd.read_csv(file_path)
    group_value = int(file_path.split('/')[-1].split('.')[0])
    df['group'] = group_value
    dfs.append(df)

group_ranking = pd.concat(dfs, ignore_index=True)

now every customer based on its age group and purchase history has rankings up to 40 products

In [23]:
group_ranking[group_ranking['customer_id']==-9223352921020755230]

,customer_id,article_id,value,rank,group
0,-9223352921020755230,568597006,115.788917,5.0,1
1,-9223352921020755230,673396002,6377.466984,1.0,1
2,-9223352921020755230,706016001,629.669042,4.0,1
3,-9223352921020755230,812167004,1222.919296,3.0,1
4,-9223352921020755230,849493006,1451.451828,2.0,1


In [24]:
group_ranking = group_ranking.merge(products[['index_group_name','article_id']], on='article_id', how='inner')
group_ranking = group_ranking.drop(['value'],axis=1)
# group_ranking = group_ranking.drop(['rank'],axis=1)

In [25]:
customers['customer_id2'] = customers['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

In [54]:
# index_rank = index_rank.merge(customers[['customer_id2','group']], left_on='customer_id',right_on='customer_id2', how='inner')

In [55]:
# index_rank = index_rank.drop(['customer_id2','group'],axis=1)

In [56]:
index_rank[index_rank['customer_id'] == -9223352921020755230]

,customer_id,garment_group_name,probability
0,-9223352921020755230,Accessories,0.1
1,-9223352921020755230,Blouses,0.2
2,-9223352921020755230,Knitwear,0.2
3,-9223352921020755230,Outdoor,0.1
4,-9223352921020755230,Trousers,0.4


In [28]:
l = index_rank.merge(group_ranking[['index_group_name','customer_id','article_id','rank','group']], on=['customer_id','index_group_name'], how='inner')

In [44]:
# l.isnull().sum()
# l = l.dropna()

In [58]:
l.loc[l['customer_id']==-9223352921020755230]

,customer_id,garment_group_name,probability,article_id,rank,group
0,-9223352921020755230,Accessories,0.1,673396002,1.0,1
1,-9223352921020755230,Blouses,0.2,849493006,2.0,1
2,-9223352921020755230,Knitwear,0.2,812167004,3.0,1
3,-9223352921020755230,Trousers,0.4,568597006,5.0,1
4,-9223352921020755230,Trousers,0.4,706016001,4.0,1


In [48]:
l.loc[l['customer_id']==-9223343869995384291]

,customer_id,index_name,probability,article_id,value,group
5,-9223343869995384291,Divided,0.040541,519583013,8.411124,3
6,-9223343869995384291,Ladies Accessories,0.047297,753692018,157.547737,3
7,-9223343869995384291,Ladies Accessories,0.047297,772565007,177.241204,3
8,-9223343869995384291,Ladieswear,0.675676,695545007,0.351746,3
9,-9223343869995384291,Ladieswear,0.675676,695632002,6.378264,3
10,-9223343869995384291,Ladieswear,0.675676,910601003,29420.736800,3
11,-9223343869995384291,Ladieswear,0.675676,762846008,1090.197544,3
12,-9223343869995384291,Ladieswear,0.675676,831611001,89.289632,3
13,-9223343869995384291,Ladieswear,0.675676,785018004,93.088166,3
14,-9223343869995384291,Ladieswear,0.675676,903926002,23261.920694,3


In [59]:
l['ranking'] = l['probability']*(21 - l['rank'])
# l['ranking'] = l['probability']*(l['value'])
l['total_rank'] = l.groupby("customer_id")['ranking'].rank("dense", ascending=False)

In [60]:
l.loc[l['customer_id']==-9223352921020755230]

,customer_id,garment_group_name,probability,article_id,rank,group,ranking,total_rank
0,-9223352921020755230,Accessories,0.1,673396002,1.0,1,2.0,5.0
1,-9223352921020755230,Blouses,0.2,849493006,2.0,1,3.8,3.0
2,-9223352921020755230,Knitwear,0.2,812167004,3.0,1,3.6,4.0
3,-9223352921020755230,Trousers,0.4,568597006,5.0,1,6.4,2.0
4,-9223352921020755230,Trousers,0.4,706016001,4.0,1,6.8,1.0


In [61]:
l = l[l['total_rank'] <= 12]

In [62]:
l.loc[l['customer_id']==-9223343869995384291]

,customer_id,garment_group_name,probability,article_id,rank,group,ranking,total_rank
8,-9223343869995384291,Blouses,0.135135,762846008,10.0,3,1.486486,10.0
9,-9223343869995384291,Blouses,0.135135,884319008,12.0,3,1.216216,11.0
17,-9223343869995384291,Jersey Fancy,0.162162,865929003,7.0,3,2.270270,5.0
18,-9223343869995384291,Jersey Fancy,0.162162,849886012,11.0,3,1.621622,9.0
19,-9223343869995384291,Jersey Fancy,0.162162,849886010,5.0,3,2.594595,2.0
20,-9223343869995384291,Jersey Fancy,0.162162,865929007,4.0,3,2.756757,1.0
21,-9223343869995384291,Jersey Fancy,0.162162,927529004,9.0,3,1.945946,6.0
23,-9223343869995384291,Knitwear,0.128378,903926002,3.0,3,2.310811,4.0
25,-9223343869995384291,Knitwear,0.128378,908292002,1.0,3,2.567568,3.0
27,-9223343869995384291,Knitwear,0.128378,821759001,6.0,3,1.925676,7.0


In [63]:
purchase_df = l.sort_values(['customer_id', 'total_rank'], ascending = False).reset_index(drop = True)
purchase_df['prediction'] = '0' + purchase_df['article_id'].astype(str) + ' '
# purchase_df = purchase_df.merge(customers[['customer_id2','group']], left_on='customer_id',right_on="customer_id2",how = 'left')
purchase_df = purchase_df.groupby('customer_id').agg({'prediction': sum}).reset_index()
purchase_df['prediction'] = purchase_df['prediction'].str.strip()
sub  = pd.read_csv('C:/Users/sepid/Desktop/H & m/sample_submission.csv',
                    usecols= ['customer_id'], 
                    dtype={'customer_id': 'string'})
sub = sub.merge(customers[['customer_id', 'group']], on='customer_id', how='inner')
sub['customer_id2'] = sub['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
sub = sub.merge(purchase_df, left_on = 'customer_id2', right_on = 'customer_id', how = 'left')
sub=sub.drop(['customer_id_y'],axis=1)
sub = sub.rename(columns={'customer_id_x': 'customer_id'})
# sub['group'] = sub['group'].astype(int)
# customers with no history will be recommended with general prediction which is top 12 quotinet
sub.loc[sub['group'] == 1, 'prediction'] = sub.loc[sub['group'] == 1, 'prediction'].fillna(gen_1)
sub.loc[sub['group'] == 2, 'prediction'] = sub.loc[sub['group'] == 2, 'prediction'].fillna(gen_2)
sub.loc[sub['group'] == 3, 'prediction'] = sub.loc[sub['group'] == 3, 'prediction'].fillna(gen_3)

sub.loc[sub['group'] == 1, 'prediction'] += ' ' + gen_1
sub.loc[sub['group'] == 2, 'prediction'] += ' ' + gen_2
sub.loc[sub['group'] == 3, 'prediction'] += ' ' + gen_3
sub['prediction'] = sub['prediction'].str.strip()
sub['prediction'] = sub['prediction'].str[:131]
sub = sub[['customer_id', 'prediction']]
sub.to_csv(f'C:/Users/sepid/Desktop/H & m/submission_.csv', index=False)

with index group:
public:0.01064
private:0.01088

with product group:
public:0.01113
private:0.01115

with germant group product:
public:0.01131
private: 0.0118